In [6]:
import pandas
from sklearn.linear_model import MultiTaskElasticNetCV, ElasticNetCV
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import normalize
from sklearn.ensemble import BaggingRegressor
from scipy.stats import pearsonr
%matplotlib inline
import matplotlib.pyplot as plt 

In [ ]:
%%timeit -r69

# this is the new benchmark script 

def low_10( df ):
    return df.sort( 'total_score' ).head( 10 )

def return_comparison_plot( ):
    df = pandas.read_csv( 'benchmarks/original.sc', sep=r'\s+' ).dropna()
    df.description = df.description.str[:-5]
    df = df.groupby( 'description' ).apply( low_10 )
    df.set_index( 'description', inplace=True )

    drop_list = [ u'all_cst', u'tot_seq_recovery', u'SR_1', u'SR_1_all_cst', u'SR_2', u'SR_2_all_cst', u'SR_3', u'SR_3_all_cst', u'SR_4', u'SR_4_all_cst', u'SR_5', u'SR_5_all_cst' ]
    df.drop( drop_list, inplace=True )

    train_set = pandas.read_csv( '../data/train_set.csv' )
    train_set.set_index( 'mutant', inplace=True )

    fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )

    constants = [ 'kcat', '1/km', 'kcat/km' ]

    for const_index, constant in enumerate( constants ): 

        X = df.join( train_set ).dropna()
        y = X[ constant ].values
        X = X.ix[ :,:'expressed' ].values

        net = ElasticNetCV( normalize=True, selection='random' )

        params_grid = {
            'cv': [ 10 ], 
            'l1_ratio': [ 0.001, 0.01, 0.1, 0.5, 0.9 ], 
        }

        grid = GridSearchCV( net, params_grid )
        bag = BaggingRegressor( base_estimator = grid )
        bag.fit( X, y )

        preds = [ bag.predict( X[i].reshape(1, -1) ) for i in range( len( X ) ) ]
        #print type( preds ), type( preds[0] ), preds
        actuals = [ y[i] for i in range( len( X ) ) ]
        #print type( actuals ), type( actuals[0] ), actuals 

        xvalues = [ preds[n][0] for n in range( len(preds ) ) ]
        yvalues = [ actuals[n] for n in range( len( actuals ) ) ]

        ax[ const_index ].scatter( yvalues, xvalues, alpha=0.4, marker='.' )
        ax[ const_index ].set_xlabel( 'Actual' )
        ax[ const_index ].set_ylabel( 'Predicted' )
        ax[ const_index ].set_title( '{}\nPCC={:.2}'.format( constant, pearsonr( xvalues, yvalues )[0] ) )

        # what do I want out? 
        # 1. the plot
        # 2. 
        
    return fig, ax

f, a = return_comparison_plot()
f.show()

/usr/local/bin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/bin/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
import sqlite3
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import BaggingRegressor

con = sqlite3.connect( 'features.db3' )
for i in [ 'interfaces', 'interface_sides' ]:
    f = pandas.read_sql_query( 'select * from {}'.format( i ), con, index_col='struct_id' )

names = pandas.read_sql_query( 'select * from structures', con, index_col='struct_id' )
feats = f.join( names ).dropna()
#print f.head()
#print names.head()
#print feats.head()

feats['shlag'] = feats.tag.str[:-10]
feats.set_index( 'shlag', inplace=True )

train_set = pandas.read_csv( '../data/train_set.csv' )
train_set.set_index( 'mutant', inplace=True )

J = feats.join( train_set ).dropna()

y = J.ix[:,'kcat':]
X = J.ix[:,'dSASA':'batch_id'] 

net = MultiTaskElasticNetCV( )
params_grid = { 'l1_ratio': [ 0.001, 0.01, 0.1, 0.5 ], 'cv': [ 10 ] }
grid = GridSearchCV( net, params_grid )
grid.fit( X, y )

print 'Model score:', grid.score( X, y )
print 'Found params:', grid.best_params_
preds = [ grid.predict( X.iloc[i].reshape( 1, -1 ) ) for i in range( len( X ) ) ]
actuals = [ y.iloc[i] for i in range( len( X ) ) ]
#print 'Pearson {}'.format( pearsonr( preds, actuals ) )

fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )
for i in range( 3 ):
    xvalues = [ preds[n][0][i] for n in range( len( preds ) ) ]
    yvalues = [ actuals[n][i] for n in range( len( actuals ) ) ]
    ax[i].scatter( yvalues, xvalues, alpha=0.4, marker='.' )
    ax[i].set_xlabel( 'Actual' )
    ax[i].set_ylabel( 'Predicted' )
    ax[i].set_title( '{}\nPCC={:.2}'.format( y.columns[i], pearsonr( xvalues, yvalues )[0] ) )
    plt.tight_layout()

#bag = BaggingRegressor( base_estimator = grid )
#bag.fit( X, y )

#net = ElasticNetCV(cv=strat)

#preds = cross_val_predict( bag, X, y=y, cv=strat )
#net.fit( X, y )
#constants = ['kcat', '1/km', 'kcat/km']
#preds = pandas.DataFrame( preds, columns=constants )
#actuals = pandas.DataFrame( y, columns=constants )
#for c in constants:
#    plt.scatter( preds[c], actuals[c] )
#    plt.show()

In [ ]:
def low_10( df ):
    return df.sort_values( by='total_score' ).head( 10 )

df = pandas.read_csv( 'benchmarks/whole.sc', sep=r'\s+' ).dropna()
df.description = df.description.str[:-5]
df = df.groupby( 'description' ).apply( low_10 )
df.set_index( 'description', inplace=True )

train_set = pandas.read_csv( '../data/train_set.csv' )
train_set.set_index( 'mutant', inplace=True )
X = df.join( train_set ).dropna()

y = X.ix[:,'kcat':]
X = X.ix[:,:'expressed'] 

net = MultiTaskElasticNetCV( selection='random', normalize=True )

params_grid = {
    'cv': [ 3, 6, 10 ], 
    'l1_ratio': [ 0.0001, 0.001, 0.01, 0.1, 0.5 ], 
}

grid = GridSearchCV( net, params_grid )
grid.fit( X, y )

print grid.score( X, y )
print grid.best_params_

preds = [ grid.predict( X.iloc[i].reshape( 1, -1 ) ) for i in range( len( X ) ) ]
actuals = [ y.iloc[i] for i in range( len( X ) ) ]

fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )
for i in range( 3 ):
    xvalues = [ preds[n][0][i] for n in range( len( preds ) ) ]
    yvalues = [ actuals[n][i] for n in range( len( actuals ) ) ]
    ax[i].scatter( yvalues, xvalues, alpha=0.4, marker='.', color='red' )
    ax[i].set_xlabel( 'Actual' )
    ax[i].set_ylabel( 'Predicted' )
    ax[i].set_title( '{}\nPCC={:.2}'.format( y.columns[i], pearsonr( xvalues, yvalues )[0] ) )
    plt.tight_layout()

In [ ]:

#X = df.join( train_set ).dropna()

#y = X.ix[:,'kcat':]
#X = X.ix[:,:'expressed'] 

#net = MultiTaskElasticNetCV( selection='random', normalize=True )

#params_grid = {
#    'cv': [ 3, 4, 5, 10, 25, 50 ], 
#    'l1_ratio': [ 0.001, 0.01, 0.1, 0.5 ], 
#}

#grid = GridSearchCV( net, params_grid )
#grid.fit( X, y )

#print grid.score( X, y )
#print grid.best_params_

#preds = [ grid.predict( X.iloc[i].reshape( 1, -1 ) ) for i in range( len( X ) ) ]
#actuals = [ y.iloc[i] for i in range( len( X ) ) ]
#print 'Pearson {}'.format( pearsonr( preds, actuals ) )

#fig, ax = plt.subplots( ncols=3, nrows=1, figsize=(10,3) )
#for i in range( 3 ):
#    xvalues = [ preds[n][0][i] for n in range( len( preds ) ) ]
#    yvalues = [ actuals[n][i] for n in range( len( actuals ) ) ]
#    ax[i].scatter( yvalues, xvalues, alpha=0.4, marker='.' )
#    ax[i].set_xlabel( 'Actual' )
#    ax[i].set_ylabel( 'Predicted' )
#   ax[i].set_title( '{}\nPCC={:.2}'.format( y.columns[i], pearsonr( xvalues, yvalues )[0] ) )
#    plt.tight_layout()

In [ ]:
from sklearn.preprocessing import normalize

df = pandas.read_csv( 'benchmarks/detect.sc', sep=r'\s+' ).dropna()
df.description = df.description.str[:-5]
df.set_index( 'description', inplace=True )
train_set = pandas.read_csv( '../data/train_set.csv' )
train_set.set_index( 'mutant', inplace=True )

constants = [ 'kcat', '1/km', 'kcat/km' ]
from sklearn.cross_validation import StratifiedKFold
for constant in constants:
    X = df.join( train_set ).dropna()
    y = X[constant].values
    X = normalize( X.ix[:,:'expressed'].values )
    strat = StratifiedKFold( y, n_folds=10 )
    net = ElasticNetCV( selection='random', normalize=True, cv=strat )

    params_grid = {
        'l1_ratio': [ 0.001, 0.01, 0.1, 0.5 ], 
    }

    grid = GridSearchCV( net, params_grid )
    grid.fit( X, y )

    print grid.score( X, y )
    print grid.best_params_

    preds = [ grid.predict( X[i].reshape( 1, -1 ) )[0] for i in range( len( X ) ) ]
    actuals = [ y.iloc[i] for i in range( len( X ) ) ]
    pcc = pearsonr( preds, actuals ) 

    xvalues = [ preds[n] for n in range( len( preds ) ) ]
    yvalues = [ actuals[n] for n in range( len( actuals ) ) ]
    plt.scatter( yvalues, xvalues, alpha=0.4, marker='.' )
    plt.xlabel( 'Actual' )
    plt.ylabel( 'Predicted' )
    plt.title( '{}\nPCC={:.2}'.format( constant, pcc[0] ) )
    plt.show()

In [ ]:
from sklearn.linear_model import MultiTaskElasticNetCV, ElasticNetCV
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import normalize
from scipy.stats import pearsonr
import pandas
%matplotlib inline
import matplotlib.pyplot as plt 

df = pandas.read_csv( 'benchmarks/detect.sc', sep=r'\s+' ).dropna()
df.description = df.description.str[:-5]
df.set_index( 'description', inplace=True )
train_set = pandas.read_csv( '../data/train_set.csv' )
train_set.set_index( 'mutant', inplace=True )
j = df.join( train_set ).dropna()
corr = j.corr()

constants = ['kcat', '1/km', 'kcat/km']
for const in constants:    
    top_4 = corr[const][ corr[const].abs() > 0.25 ] #get( 'kcat' )
    print top_4
    
